# Web Scraping with Beautiful Soup

This tutorial will provide a very basic introduction to using the Beautiful Soup package to scrape text data from the web. 

## Installation

In a terminal, install Beautiful Soup if necessary by running <pre><code>conda install beautiful-soup</code></pre>

## Retrieving HTML

The general idea behind web scraping is to retrieve data that exists on a website, and convert it into a format that is usable for analysis. Webpages are rendered by the brower from HTML and CSS code, but much of the information included in the HTML underlying any website is not interesting to us.

We begin by reading in the source code for a given web page and creating a Beautiful Soup object with the BeautifulSoup function.

### urllib2
urllib2 is a module for working with urls, we will use it to open connections to urls and retrieve the webpage source.

In [2]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


The soup object contains all of the HTML in the original document.

In [3]:
soup.prettify()[0:1000]

u'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml">\n <head>\n  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">\n  </script>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>\n  <meta content="#CC0000" name="theme-color"/>\n  <script type="text/javascript">\n   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {\n        window.location = \'http://a.m.espn.go.com/mobilecache/general/apps/sc\';\n    }\n  </script>\n  <title>\n   2015-16 Regular Season NBA Player Stats and League Leaders - Scoring Per Game - National Basketball Association - ESPN\n  </title>\n  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>\n  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>\n  <meta content="noodp" name="googlebot"/>\n  <meta co

## Parsing HTML

By "parsing HTML", we mean pulling out only the relevant tags/attributes for our analysis.  What Beautiful Soup does is provide a handy bunch of methods for doing this efficiently.

### find method

The find method will search for and return the first tag matching your corresponding search criteria, if it exists.  You can specify tag and attribute info etc.  There is also a findAll method that will return a collection of tags matching your query.

In [4]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Points Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
       PTS

Notice that the object returned by the find method is just another inner HTML structure, which we can step through just like we would have with the original soup object.  We've gone to the location in the webpage where the table that we seek starts, now we can use find again to get to the table data.

In [5]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/play

### Searching by Attributes

Now that we have the table object, we need to step through the rows.  First we'll find the header row so we can populate what the field names will be in our data.  Here we're searching for tags under the table tag whose class attritbute is "colhead".  

In [6]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">
   PTS
  </a>
 </td>
 <td>
  <span title="Field Goals Made-Attempted Per Game">
   FGM-FGA
  </span>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">
   FG%
  </a>
 </td>
 <td>
  <span title="3-Point Field Goals Made-Attempted Per Game">
   3PM-3PA
  </span>
 </td>
 <t

Now we find the actual values by searching for the 'td' tags, which is the tag for table data.

In [7]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td>, <td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td>, <td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qualified/false" t

Finally, we step through these columns and save them to a list to be used later.  We'll ignore the rank column (RK) because that doesn't give us anything we want later.  We also separate the **PLAYER** column into **PLAYER** and **POSITION**.

In [8]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'PTS', u'FGM-FGA', u'FG%', u'3PM-3PA', u'3P%', u'FTM-FTA', u'FT%']


### Stepping Through a Table

The table rows are indicated by the tag 'tr'.  Again we can find them all and iterate through them.  Within each row we iterate through the respective columns.

In [9]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/order/false" title="Points Per Game">PTS</a></td><td><span title="Field Goals Made-Attempted Per Game">FGM-FGA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/fieldGoalPct/qualified/false" title="Field Goal Percentage">FG%</a></td><td><span title="3-Point Field Goals Made-Attempted Per Game">3PM-3PA</span></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/scoring-per-game/sort/threePointFieldGoalPct/qu

We will save our results in 2 different ways to demonstrate how we can handle both.  The first will be a list of dicts where the key is the field name and the value is the field value.  The second will just be a list of lists of stats with no field name values (we've already defined them earlier).

In [10]:
players_stats_dicts = []
players_stats_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_stats = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_stats.append(player_name)
    player_stats.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_stats.append(stat)
    players_stats_array.append(player_stats)
    player_stats = dict(zip(cols, player_stats))
    players_stats_dicts.append(player_stats)
print players_stats_dicts[0:5]        

[{u'FGM-FGA': u'11.1-21.5', u'MPG': u'35.1', u'FTM-FTA': u'6.0-6.5', u'FG%': u'.517', u'GP': u'11', u'PLAYER': u'Stephen Curry', u'FT%': u'.930', u'TEAM': u'GS', u'3PM-3PA': u'5.2-11.5', 'POSITION': u'PG', u'PTS': u'33.4', u'3P%': u'.452'}, {u'FGM-FGA': u'7.9-21.3', u'MPG': u'39.4', u'FTM-FTA': u'10.3-11.9', u'FG%': u'.371', u'GP': u'10', u'PLAYER': u'James Harden', u'FT%': u'.866', u'TEAM': u'HOU', u'3PM-3PA': u'2.3-9.6', 'POSITION': u'SG', u'PTS': u'28.4', u'3P%': u'.240'}, {u'FGM-FGA': u'9.6-19.1', u'MPG': u'34.8', u'FTM-FTA': u'6.3-7.0', u'FG%': u'.503', u'GP': u'8', u'PLAYER': u'Kevin Durant', u'FT%': u'.893', u'TEAM': u'OKC', u'3PM-3PA': u'2.6-5.9', 'POSITION': u'SF', u'PTS': u'28.1', u'3P%': u'.447'}, {u'FGM-FGA': u'10.5-20.8', u'MPG': u'36.3', u'FTM-FTA': u'4.9-8.1', u'FG%': u'.505', u'GP': u'10', u'PLAYER': u'LeBron James', u'FT%': u'.605', u'TEAM': u'CLE', u'3PM-3PA': u'1.1-4.2', 'POSITION': u'SF', u'PTS': u'27.0', u'3P%': u'.262'}, {u'FGM-FGA': u'8.3-19.7', u'MPG': u'30.3', 

Here we've used the zip function to combine pairs of lists into tuples, and then transformed that into a dict to get a dictionary of FIELD --> VALUE for every player in the table.

Beautiful Soup has many other features, including the ability to step up, down, and sideways in the HTML tree and basically search for any tags, attributes, or values.  For more, take a look at the [Documentation](http://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Load into Pandas

Let's load our scraped data into Pandas and take a look at it.  Here is the first way we can do it, simply directly from the dictionary we defined.

In [11]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.452,5.2-11.5,.517,11.1-21.5,.930,6.0-6.5,11,35.1,Stephen Curry,PG,33.4,GS
1,.240,2.3-9.6,.371,7.9-21.3,.866,10.3-11.9,10,39.4,James Harden,SG,28.4,HOU
2,.447,2.6-5.9,.503,9.6-19.1,.893,6.3-7.0,8,34.8,Kevin Durant,SF,28.1,OKC
3,.262,1.1-4.2,.505,10.5-20.8,.605,4.9-8.1,10,36.3,LeBron James,SF,27.0,CLE
4,.435,1.7-3.8,.424,8.3-19.7,.758,8.3-11.0,6,30.3,DeMarcus Cousins,PF,26.7,SAC


Now here is a 2nd way we can do it.  We convert the 2D stats array into a numpy array and create a Pandas dataframe from it along with the list of column headers we defined earlier.

In [12]:
np_array = np.array(players_stats_array)
df = pd.DataFrame(np_array, columns=cols)
df.head()

,PLAYER,POSITION,TEAM,GP,MPG,PTS,FGM-FGA,FG%,3PM-3PA,3P%,FTM-FTA,FT%
0,Stephen Curry,PG,GS,11,35.1,33.4,11.1-21.5,.517,5.2-11.5,.452,6.0-6.5,.930
1,James Harden,SG,HOU,10,39.4,28.4,7.9-21.3,.371,2.3-9.6,.240,10.3-11.9,.866
2,Kevin Durant,SF,OKC,8,34.8,28.1,9.6-19.1,.503,2.6-5.9,.447,6.3-7.0,.893
3,LeBron James,SF,CLE,10,36.3,27.0,10.5-20.8,.505,1.1-4.2,.262,4.9-8.1,.605
4,DeMarcus Cousins,PF,SAC,6,30.3,26.7,8.3-19.7,.424,1.7-3.8,.435,8.3-11.0,.758


## Exercise

The goal of this exercise is to combine the scoring and assists statistics for every player in the NBA in 2014-2015.  The end result will have them in a pandas dataframe with the fields from both pages for every player.

The general steps should be as follows:
- Create a function get_cols that retrieves the names of the header columns given a table element (skip the ranks, split the positions)
- Create a function get_data that retrieves the actual table data given a table element (skip the ranks, split the positions).  You can use either the dict approach or the numpy array approach.
- Write a python loop to loop through the various pages and call these functions on the appropriate urls so that you can retrieve every player (rather than just the top few).
- Repeat the above on both the scoring and assists URLs to get a pandas dataframe for both of them
- Use the pandas.DataFrame.join() function to join your 2 pandas dataframes together and get a total result

In [13]:
from bs4 import BeautifulSoup
import urllib2
page = urllib2.urlopen('http://espn.go.com/nba/statistics/player/_/stat/assists/qualified/false').read()
soup = BeautifulSoup(page)
print type(soup)

<class 'bs4.BeautifulSoup'>


In [14]:
soup.prettify()[0:1000]

u'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml">\n <head>\n  <script src="http://sports-ak.espn.go.com/sports/optimizely.js">\n  </script>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <link href="http://a.espncdn.com/prod/assets/icons/E.svg" mask="" rel="icon" sizes="any"/>\n  <meta content="#CC0000" name="theme-color"/>\n  <script type="text/javascript">\n   if(true && navigator && navigator.userAgent.toLowerCase().indexOf("teamstream") >= 0) {\n        window.location = \'http://a.m.espn.go.com/mobilecache/general/apps/sc\';\n    }\n  </script>\n  <title>\n   2015-16 Regular Season NBA Player Stats and League Leaders - Assists - National Basketball Association - ESPN\n  </title>\n  <meta content="xuj1ODRluWa0frM-BjIr_aSHoUC7HB5C1MgmYAM_GkA" name="google-site-verification"/>\n  <meta content="B1FEB7C682C46C8FCDA3130F3D18AC28" name="msvalidate.01"/>\n  <meta content="noodp" name="googlebot"/>\n  <meta content="in

In [15]:
table_div = soup.find(id='my-players-table')
print table_div.prettify()[0:4000]

<div class="col-main" id="my-players-table">
 <div class="mod-container mod-table">
  <div class="mod-header stathead">
   <h4>
    Assists Per Game Leaders - All Players
   </h4>
  </div>
  <div class="mod-content">
   <table cellpadding="3" cellspacing="1" class="tablehead">
    <tr align="right" class="colhead">
     <td align="left" style="width:20px;">
      RK
     </td>
     <td align="left">
      PLAYER
     </td>
     <td align="left">
      TEAM
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/gamesPlayed/qualified/false" title="Games Played">
       GP
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgMinutes/qualified/false" title="Minutes Per Game">
       MPG
      </a>
     </td>
     <td>
      <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/assists/qualified/false" title="Assists">
       AST
      </a>
     </td>
     <td>
      <a href="

In [16]:
table = table_div.find("table")
print table

<table cellpadding="3" cellspacing="1" class="tablehead">
<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/assists/qualified/false" title="Assists">AST</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/order/false" title="Assists Per Game">APG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/turnovers/qualified/false" title="Turnovers">TO</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgTurnovers/qualified/false" title="Turnovers Per Gam

In [17]:
table_head = table.find(attrs={"class":'colhead'})
print table_head.prettify()

<tr align="right" class="colhead">
 <td align="left" style="width:20px;">
  RK
 </td>
 <td align="left">
  PLAYER
 </td>
 <td align="left">
  TEAM
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/gamesPlayed/qualified/false" title="Games Played">
   GP
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgMinutes/qualified/false" title="Minutes Per Game">
   MPG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/assists/qualified/false" title="Assists">
   AST
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/order/false" title="Assists Per Game">
   APG
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/turnovers/qualified/false" title="Turnovers">
   TO
  </a>
 </td>
 <td>
  <a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgTurnovers/qua

In [18]:
header_cols = table_head.findAll('td')
print header_cols

[<td align="left" style="width:20px;">RK</td>, <td align="left">PLAYER</td>, <td align="left">TEAM</td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/assists/qualified/false" title="Assists">AST</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/order/false" title="Assists Per Game">APG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/turnovers/qualified/false" title="Turnovers">TO</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgTurnovers/qualified/false" title="Turnovers Per Game">TOPG</a></td>, <td><a href="http://espn.go.com/nba/statistics/player/_/s

In [19]:
cols = []
for header_col in header_cols:
    val = header_col.string
    if val != 'RK':
        cols.append(val)
    if val == 'PLAYER':
        cols.append('POSITION')
print cols

[u'PLAYER', 'POSITION', u'TEAM', u'GP', u'MPG', u'AST', u'APG', u'TO', u'TOPG', u'AP48M', u'AST/TO']


In [20]:
table_rows = table.findAll('tr')
print table_rows

[<tr align="right" class="colhead"><td align="left" style="width:20px;">RK</td><td align="left">PLAYER</td><td align="left">TEAM</td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/gamesPlayed/qualified/false" title="Games Played">GP</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgMinutes/qualified/false" title="Minutes Per Game">MPG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/assists/qualified/false" title="Assists">AST</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgAssists/qualified/false/order/false" title="Assists Per Game">APG</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/turnovers/qualified/false" title="Turnovers">TO</a></td><td><a href="http://espn.go.com/nba/statistics/player/_/stat/assists/sort/avgTurnovers/qualified/false" title="Turnovers Per Game">TOPG</a></td><td><a href="http://espn.go.com/nba/stati

In [21]:
players_assists_dicts = []
players_assists_array = []
for row in table_rows:
    if row.attrs['class'][0]=='colhead':
        continue
    player_assists = []
    row_cols = row.find_all('td')
    col_vals = []
    player_col = row_cols[1]
    player_name = player_col.find('a').string
    player_position = player_col.contents[1]
    player_position = player_position.split(' ')[1]
    player_assists.append(player_name)
    player_assists.append(player_position)
    for i in range(2, len(row_cols)):
        stat = row_cols[i].string
        player_assists.append(stat)
    players_assists_array.append(player_assists)
    player_assists = dict(zip(cols, player_assists))
    players_assists_dicts.append(player_assists)
print players_assists_dicts[0:5]   

[{u'MPG': u'34.7', u'GP': u'9', u'AST': u'98', u'PLAYER': u'Russell Westbrook', u'TO': u'45', u'AP48M': u'15.1', u'TEAM': u'OKC', u'TOPG': u'5.0', 'POSITION': u'PG', u'APG': u'10.9', u'AST/TO': u'2.18'}, {u'MPG': u'34.7', u'GP': u'10', u'AST': u'91', u'PLAYER': u'Rajon Rondo', u'TO': u'36', u'AP48M': u'12.6', u'TEAM': u'SAC', u'TOPG': u'3.6', 'POSITION': u'PG', u'APG': u'9.1', u'AST/TO': u'2.53'}, {u'MPG': u'30.8', u'GP': u'6', u'AST': u'54', u'PLAYER': u'Ricky Rubio', u'TO': u'12', u'AP48M': u'14.0', u'TEAM': u'MIN', u'TOPG': u'2.0', 'POSITION': u'PG', u'APG': u'9.0', u'AST/TO': u'4.50'}, {u'MPG': u'33.6', u'GP': u'8', u'AST': u'68', u'PLAYER': u'John Wall', u'TO': u'34', u'AP48M': u'12.1', u'TEAM': u'WSH', u'TOPG': u'4.3', 'POSITION': u'PG', u'APG': u'8.5', u'AST/TO': u'2.00'}, {u'MPG': u'32.6', u'GP': u'7', u'AST': u'56', u'PLAYER': u'Chris Paul', u'TO': u'20', u'AP48M': u'11.8', u'TEAM': u'LAC', u'TOPG': u'2.9', 'POSITION': u'PG', u'APG': u'8.0', u'AST/TO': u'2.80'}]


In [22]:
import pandas as pd
import numpy as np

df = pd.DataFrame.from_dict(players_assists_dicts)
df.head()

,AP48M,APG,AST,AST/TO,GP,MPG,PLAYER,POSITION,TEAM,TO,TOPG
0,15.1,10.9,98,2.18,9,34.7,Russell Westbrook,PG,OKC,45,5.0
1,12.6,9.1,91,2.53,10,34.7,Rajon Rondo,PG,SAC,36,3.6
2,14.0,9.0,54,4.50,6,30.8,Ricky Rubio,PG,MIN,12,2.0
3,12.1,8.5,68,2.00,8,33.6,John Wall,PG,WSH,34,4.3
4,11.8,8.0,56,2.80,7,32.6,Chris Paul,PG,LAC,20,2.9


In [25]:
import pandas as pd
pd.merge(players_assists_dicts, players_stats_dicts, on='PLAYER', how='inner')

TypeError: list indices must be integers, not str

In [28]:
import pandas as pd
pd.merge(players_assists_dataframe, players_stats, on='PLAYER', how='inner')

NameError: name 'players_assists_dataframe' is not defined

In [30]:
df = pd.DataFrame.from_dict(players_assists_dicts)
df.head()

,AP48M,APG,AST,AST/TO,GP,MPG,PLAYER,POSITION,TEAM,TO,TOPG
0,15.1,10.9,98,2.18,9,34.7,Russell Westbrook,PG,OKC,45,5.0
1,12.6,9.1,91,2.53,10,34.7,Rajon Rondo,PG,SAC,36,3.6
2,14.0,9.0,54,4.50,6,30.8,Ricky Rubio,PG,MIN,12,2.0
3,12.1,8.5,68,2.00,8,33.6,John Wall,PG,WSH,34,4.3
4,11.8,8.0,56,2.80,7,32.6,Chris Paul,PG,LAC,20,2.9


In [31]:
df = pd.DataFrame.from_dict(players_stats_dicts)
df.head()

,3P%,3PM-3PA,FG%,FGM-FGA,FT%,FTM-FTA,GP,MPG,PLAYER,POSITION,PTS,TEAM
0,.452,5.2-11.5,.517,11.1-21.5,.930,6.0-6.5,11,35.1,Stephen Curry,PG,33.4,GS
1,.240,2.3-9.6,.371,7.9-21.3,.866,10.3-11.9,10,39.4,James Harden,SG,28.4,HOU
2,.447,2.6-5.9,.503,9.6-19.1,.893,6.3-7.0,8,34.8,Kevin Durant,SF,28.1,OKC
3,.262,1.1-4.2,.505,10.5-20.8,.605,4.9-8.1,10,36.3,LeBron James,SF,27.0,CLE
4,.435,1.7-3.8,.424,8.3-19.7,.758,8.3-11.0,6,30.3,DeMarcus Cousins,PF,26.7,SAC


In [32]:
#we want to join these two!! but how? realize that these are stil head lists (not full data) but still want to join

In [39]:
pandas.DataFrame.join(players_stats_dicts, players_assists_dicts)

NameError: name 'pandas' is not defined

In [41]:
players_stats_dicts.join(other=players_assists_dicts,on='player',how='inner')

AttributeError: 'list' object has no attribute 'join'